# MLflow Prompt Registryを使う

https://docs.databricks.com/aws/ja/mlflow3/genai/prompt-version-mgmt/prompt-registry/reference


> **MLflow Prompt Registry**は、生成AI（GenAI）アプリケーションにおけるプロンプトエンジニアリングと管理を効率化する強力なツールです。組織内でプロンプトをバージョン管理・追跡・再利用できるため、プロンプト開発における一貫性の維持やコラボレーションの向上に役立ちます。
> 
> - **再利用性**：プロンプトを一元管理し、複数のアプリケーションで再利用
> - **バージョン管理**：Git風のコミットベースのバージョン管理と、差分比較（diffハイライト）によるプロンプトバージョンの進化の追跡
> - **エイリアス**：柔軟なデプロイパイプラインを構築し、プロンプトバージョンをアプリケーション本体から分離。A/Bテストやロールバックも容易
> - **系譜（Lineage）**：MLflowの既存機能（モデル追跡や評価）とシームレスに連携し、GenAIライフサイクル全体を管理
> - **コラボレーション**：組織全体でプロンプトを共有し、チーム間でのプロンプト開発を促進
> 
> MLflow Prompt RegistryのDatabricks Unity Catalogとの連携は近日公開予定です。
> 
> ---
> 
> ## Unity Catalogとの統合
> 
> Prompt Registryは、Databricks Unity Catalogのガバナンスフレームワークを活用し、以下の機能を提供します。
> 
> - **一元化されたプロンプト管理**：プロンプトテンプレートを統制・検索可能なレジストリで管理
> - **バージョン管理**：プロンプトの変更履歴を完全な系譜とロールバック機能で追跡
> - **アクセス制御**：Unity Catalogの権限システムを利用し、プロンプトの閲覧・編集・利用を制御
> - **コラボレーション**：チーム間でのプロンプト開発の共有と協働を可能に
> - **ガバナンス**：分類やコンプライアンス要件を含むデータガバナンスポリシーをプロンプトテンプレートに適用
> 
> ---
> 
> ### 詳細なアクセス制御
> 
> - **個別・チーム・組織単位でのアクセス制御**
> - **データの系譜追跡**：プロンプトが異なるアプリケーションや実験でどのように利用されているかを追跡
> - **監査証跡**：プロンプトへのアクセスや変更を監視
> - **分類とタグ付け**：メタデータでプロンプトを整理し、ガバナンスポリシーを適用
> - **ワークスペース間での安全な共有**
> 
> ---
> 
> ## 今後の機能
> 
> 今後、Prompt Registryでは以下の機能がサポートされる予定です。
> 
> - **テンプレート管理**：変数置換機能付きでプロンプトテンプレートを作成・編集・バージョン管理
> - **テストと検証**：デプロイ前に評価データセットでプロンプトをテスト
> - **A/Bテスト**：異なるプロンプトバージョンを比較し、パフォーマンスを最適化
> - **MLflow Trackingとの連携**：プロンプトバージョンと実験・モデル実行を自動的に関連付け
> - **APIアクセス**：自動化ワークフローでプロンプトをプログラムから利用


In [0]:
%pip install -U -q "mlflow[databricks]>=3.1.0" databricks-langchain openai

## Original Tutorial

### プロンプトの登録

In [0]:
import mlflow

catalog = "workspace"
schema = "default"
prompt_name = "sample_prompt"
uc_prompt_name = f"{catalog}.{schema}.{prompt_name}"

# Register a prompt template
prompt = mlflow.genai.register_prompt(
    name=uc_prompt_name,
    template="あなたは親切なアシスタントです。次の問いに答えてください: {{question}}",
    commit_message="Initial Commit"
)
print(f"Created version {prompt.version}")  # "Created version 1"

### プロンプトの取得・検索

In [0]:
prompt = mlflow.genai.load_prompt(name_or_uri=f"prompts:/{uc_prompt_name}/1")

prompt

In [0]:
prompts = mlflow.genai.search_prompts(filter_string=f"catalog='{catalog}' and schema='{schema}'")
prompts


### プロンプトの利用

In [0]:
import mlflow
from openai import OpenAI

# MLflowの自動ロギングを有効にして、アプリケーションにトレースを追加
mlflow.openai.autolog()

# 実行ノートブックと同じ資格情報を使用してOpenAIクライアント経由でDatabricks LLMに接続
mlflow_creds = mlflow.utils.databricks_utils.get_databricks_host_creds()
client = OpenAI(
    api_key=mlflow_creds.token,
    base_url=f"{mlflow_creds.host}/serving-endpoints"
)

# レジストリからプロンプトをロード
prompt = mlflow.genai.load_prompt(f"prompts:/{uc_prompt_name}/1")
formatted_prompt = prompt.format(question="Databricksとは何ですか?")

# LLMを呼び出す
response = client.chat.completions.create(
    model="databricks-llama-4-maverick",
    messages=[
        {
            "role": "user",
            "content": formatted_prompt,
        },
    ],
)
response.choices[0].message.content


In [0]:
import mlflow
from databricks_langchain import ChatDatabricks
from langchain_core.prompts import ChatPromptTemplate
import codecs

# MLflowの自動ロギングを有効化
mlflow.langchain.autolog()

prompt = mlflow.genai.load_prompt(name_or_uri=f"prompts:/{uc_prompt_name}/1")

# LangChainのプロンプトテンプレートをMLflowのPromptから作成する
langchain_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "user",
            codecs.decode(prompt.to_single_brace_format(), "unicode-escape"),
        ),
    ]
)

llm = ChatDatabricks(model="databricks-llama-4-maverick")
chain = langchain_prompt | llm

# チェーンを実行
chain.invoke({"question": "Databricksの特徴を教えてください"})

### エイリアスを設定

In [0]:
import mlflow
mlflow.genai.set_prompt_alias(
    name=f"{uc_prompt_name}",
    alias="production",
    version=1
)

In [0]:
mlflow.genai.load_prompt(f"prompts:/{uc_prompt_name}@production")


## Quick Start

In [0]:
import mlflow
from databricks_langchain import ChatDatabricks

mlflow.langchain.autolog()

uc_name = "workspace.default.customer_support"

# Register a prompt template
prompt = mlflow.genai.register_prompt(
    name=uc_name,
    template="あなたは親切なアシスタントです。次の問いに答えてください: {{question}}",
    commit_message="初期カスタマーアシスタントプロンプト"
)
print(f"Created version {prompt.version}")  # "Created version 1"

# Set a production alias
mlflow.genai.set_prompt_alias(
    name=uc_name,
    alias="production",
    version=1
)

llm = ChatDatabricks(model="databricks-llama-4-maverick")

# Load and use the prompt in your application
prompt = mlflow.genai.load_prompt(name_or_uri=f"prompts:/{uc_name}/1")
response = llm.invoke(prompt.format(question="How do I reset my password?"))

# アプリバージョンとの紐づけ

In [0]:
import mlflow

# Replace with a Unity Catalog schema where you have CREATE FUNCTION permission
uc_schema = "workspace.default"
prompt_name = "customer_support_prompt"

# Define the prompt template with variables
initial_template = """\
You are a helpful customer support assistant for {{company_name}}.

Please help the customer with their inquiry about: {{topic}}

Customer Question: {{question}}

Provide a friendly, professional response that addresses their concern.
"""

# Register a new prompt
prompt = mlflow.genai.register_prompt(
    name=f"{uc_schema}.{prompt_name}",
    template=initial_template,
    commit_message="Initial customer support prompt",
    # version_metadata={
    #     "author": "support-team@company.com",
    #     "use_case": "customer_service"
    # },
    tags={
        "department": "customer_support",
        "language": "en"
    }
)

print(f"Created prompt '{prompt.name}' (version {prompt.version})")

In [0]:
mlflow.delete_log

In [0]:
import mlflow
import subprocess
from openai import OpenAI

# MLflowの自動ロギングを有効にして、アプリケーションにトレースを追加
mlflow.openai.autolog()

# 同じ資格情報を使用してOpenAI経由でDatabricks LLMに接続
# 代わりに独自のOpenAI資格情報を使用することも可能
mlflow_creds = mlflow.utils.databricks_utils.get_databricks_host_creds()
client = OpenAI(
    api_key=mlflow_creds.token,
    base_url=f"{mlflow_creds.host}/serving-endpoints"
)

# アプリケーションとそのバージョン識別子を定義
app_name = "customer_support_agent"

# バージョニングのために現在のgitコミットハッシュを取得
try:
    git_commit = (
        subprocess.check_output(["git", "rev-parse", "HEAD"])
        .decode("ascii")
        .strip()[:8]
    )
    version_identifier = f"git-{git_commit}"
except subprocess.CalledProcessError:
    version_identifier = "local-dev"  # gitリポジトリにない場合のフォールバック
logged_model_name = f"{app_name}-{version_identifier}"

# アクティブなモデルコンテキストを設定 - これにより、このバージョンのアプリケーションを表すLoggedModelが作成される
active_model_info = mlflow.set_active_model(name=logged_model_name)
print(
    f"Active LoggedModel: '{active_model_info.name}', Model ID: '{active_model_info.model_id}'"
)

# アプリケーションパラメータをログ
# これらのパラメータは、このアプリケーションバージョンの構成を追跡するのに役立つ
app_params = {
    "llm": "databricks-llama-4-maverick",
    "temperature": 0.7,
    "max_tokens": 500
}
# mlflow.log_model_params(model_id=active_model_info.model_id, params=app_params)

# レジストリからプロンプトをロード
# 注意: set_active_model()を呼び出した後にプロンプトをロードすることが、プロンプトバージョンとLoggedModelの間の自動系譜追跡を可能にする
prompt = mlflow.genai.load_prompt(f"prompts:/{uc_schema}.{prompt_name}/1")
print(f"Loaded prompt version {prompt.version}")

# トレースデコレータを使用してアプリケーションのエントリポイントをキャプチャ
# この関数によって作成された各トレースは、上記で設定したLoggedModel（アプリケーションバージョン）に自動的にリンクされる。さらに、LoggedModelはレジストリからロードされたプロンプトバージョンにリンクされる
@mlflow.trace
def customer_support_app(company_name: str, topic: str, question: str):
    # 変数でプロンプトをフォーマット
    formatted_prompt = prompt.format(
        company_name=company_name,
        topic=topic,
        question=question
    )

    # LLMを呼び出す
    response = client.chat.completions.create(
        model="databricks-llama-4-maverick",  # モデルを置き換える
        messages=[
            {
                "role": "user",
                "content": formatted_prompt,
            },
        ],
        temperature=0.7,
        max_tokens=500
    )
    return response.choices[0].message.content

# アプリケーションをテスト
result = customer_support_app(
    company_name="TechCorp",
    topic="billing",
    question="I was charged twice for my subscription last month. Can you help?"
)
print(f"\nResponse: {result}")

In [0]:
# Create an improved version of the prompt
improved_template = """\
You are a helpful and empathetic customer support assistant for {{company_name}}.

Customer Topic: {{topic}}
Customer Question: {{question}}

Please provide a response that:
1. Acknowledges the customer's concern with empathy
2. Provides a clear solution or next steps
3. Offers additional assistance if needed
4. Maintains a friendly, professional tone

Remember to:
- Use the customer's name if provided
- Be concise but thorough
- Avoid technical jargon unless necessary
"""

# Register the new version
updated_prompt = mlflow.genai.register_prompt(
    name=f"{uc_schema}.{prompt_name}",
    template=improved_template,
    commit_message="Added structured response guidelines for better customer experience",
    # version_metadata={
    #     "author": "support-team@company.com",
    #     "improvement": "Added empathy guidelines and response structure"
    # }
)

print(f"Created version {updated_prompt.version} of '{updated_prompt.name}'")

In [0]:
# Create a new application version
new_version_identifier = "v2-improved-prompt"
new_logged_model_name = f"{app_name}-{new_version_identifier}"

# Set the new active model
active_model_info_v2 = mlflow.set_active_model(name=new_logged_model_name)
print(
    f"Active LoggedModel: '{active_model_info_v2.name}', Model ID: '{active_model_info_v2.model_id}'"
)

# Log updated parameters
app_params_v2 = {
    "llm": "databricks-llama-4-maverick",
    "temperature": 0.7,
    "max_tokens": 500,
    "prompt_version": "2"  # Track which prompt version we're using
}
mlflow.log_model_params(model_id=active_model_info_v2.model_id, params=app_params_v2)

# Load the new prompt version
prompt_v2 = mlflow.genai.load_prompt(f"prompts:/{uc_schema}.{prompt_name}/2")

# Update the app to use the new prompt
@mlflow.trace
def customer_support_app_v2(company_name: str, topic: str, question: str):
    # Format the prompt with variables
    formatted_prompt = prompt_v2.format(
        company_name=company_name,
        topic=topic,
        question=question
    )

    # Call the LLM
    response = client.chat.completions.create(
        model="databricks-llama-4-maverick",
        messages=[
            {
                "role": "user",
                "content": formatted_prompt,
            },
        ],
        temperature=0.7,
        max_tokens=500
    )
    return response.choices[0].message.content

# Test with the same question to see the difference
result_v2 = customer_support_app_v2(
    company_name="TechCorp",
    topic="billing",
    question="I was charged twice for my subscription last month. Can you help?"
)
print(f"\nImproved Response: {result_v2}")